In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content
!git clone https://github.com/Rutgers-ECE-MML4SS/multimodal-sensor-fusion-with-attention-saharrgl.git
%cd multimodal-sensor-fusion-with-attention-saharrgl

/content
Cloning into 'multimodal-sensor-fusion-with-attention-saharrgl'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 55 (delta 13), reused 7 (delta 7), pack-reused 27 (from 2)
Receiving objects: 100% (55/55), 58.74 KiB | 14.68 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/multimodal-sensor-fusion-with-attention-saharrgl


In [3]:
%cd /content/multimodal-sensor-fusion-with-attention-saharrgl

/content/multimodal-sensor-fusion-with-attention-saharrgl


In [4]:
!ls

analysis  config  environment.yml  experiments	README.md  runs  src  tests


In [5]:
!sed -n '1,200p' environment.yml

name: a2
channels:
  - pytorch
  - conda-forge
  - defaults
dependencies:
  # Core
  - python=3.10
  - pip

  # PyTorch (CPU version for compatibility)
  - pytorch=2.0.1
  - torchvision=0.15.2
  - torchaudio=2.0.2
  - cpuonly

  # Scientific computing
  - numpy=1.24.3
  - scipy=1.10.1
  - pandas=2.0.2

  # Machine learning
  - scikit-learn=1.2.2

  # Configuration management
  - hydra-core=1.3.2
  - omegaconf=2.3.0

  # Experiment tracking & logging
  - tensorboard=2.13.0
  - tqdm=4.65.0

  # Visualization
  - matplotlib=3.7.1
  - seaborn=0.12.2

  # Testing
  - pytest=7.3.2
  - pytest-cov=4.1.0

  # Pip packages
  - pip:
      # PyTorch Lightning for training
      - pytorch-lightning==2.0.3
      - lightning==2.0.3
      
      # Data handling
      - h5py==3.9.0
      
      # Additional utilities
      - einops==0.6.1  # Tensor operations
      - timm==0.9.2    # If using vision models



In [6]:
!pip install numpy pandas scipy matplotlib seaborn tqdm
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install einops pytest scikit-learn

Looking in indexes: https://download.pytorch.org/whl/cu121


In [7]:
!pip install transformers

In [8]:
import torch
import numpy as np
import einops

print("✅ All core libraries loaded successfully!")

✅ All core libraries loaded successfully!


In [9]:
!pytest -q

.........................                                                [100%]
25 passed in 2.42s


In [10]:
!pip install pytorch-lightning==2.1.3 lightning-utilities

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 65.6 MB/s eta 0:00:00


In [11]:
import pytorch_lightning as pl
print("✅ PyTorch Lightning Installed Successfully!")

✅ PyTorch Lightning Installed Successfully!


In [12]:
!pip install hydra-core==1.3.2 omegaconf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.7 MB/s eta 0:00:00


In [13]:
!python src/train.py

Configuration:
dataset:
  name: pamap2
  data_dir: ./data
  modalities:
  - imu_hand
  - imu_chest
  - imu_ankle
  - heart_rate
  train_split: 0.7
  val_split: 0.15
  test_split: 0.15
  batch_size: 32
  num_workers: 4
model:
  fusion_type: hybrid
  hidden_dim: 256
  output_dim: 128
  num_heads: 4
  dropout: 0.1
  encoders:
    video:
      type: frame
      input_dim: 512
      temporal_pooling: attention
    imu:
      type: sequence
      input_dim: 64
      encoder_type: lstm
      num_layers: 2
training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.1
evaluation:
  metrics:
  - accuracy
  - f1_macro
  - ece
  - nll
  missing_modality_test: true
  uncertainty_analysis: true
  num_calibration_bins: 15
uncertainty:
  method: dropout
  num_mc_samples: 10
  temperature

In [14]:
!pytest -v

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/multimodal-sensor-fusion-with-attention-saharrgl
plugins: hydra-core-1.3.2, typeguard-4.4.4, langsmith-0.4.41, anyio-4.11.0
collected 25 items                                                             

tests/test_attention.py::TestCrossModalAttention::test_output_shape PASSED [  4%]
tests/test_attention.py::TestCrossModalAttention::test_with_mask PASSED  [  8%]
tests/test_attention.py::TestCrossModalAttention::test_gradient_flow PASSED [ 12%]
tests/test_attention.py::TestTemporalAttention::test_output_shape PASSED [ 16%]
tests/test_attention.py::TestTemporalAttention::test_with_mask PASSED    [ 20%]
tests/test_attention.py::TestPairwiseModalityAttention::test_output_structure PASSED [ 24%]
tests/test_encoders.py::TestSequenceEncoder::test_lstm_encoder_shape PASSED [ 28%]
tests/test_

In [15]:
# Edit src/fusion.py
# Test with:
!python -m pytest /content/multimodal-sensor-fusion-with-attention-saharrgl/tests/test_fusion.py::TestFusionInterfaces::test_early_fusion_shape -v

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/multimodal-sensor-fusion-with-attention-saharrgl
plugins: hydra-core-1.3.2, typeguard-4.4.4, langsmith-0.4.41, anyio-4.11.0
collected 1 item                                                               

tests/test_fusion.py::TestFusionInterfaces::test_early_fusion_shape PASSED [100%]

============================== 1 passed in 1.79s ===============================


In [16]:
# Edit src/encoders.py
# Test with:
!python -m pytest /content/multimodal-sensor-fusion-with-attention-saharrgl/tests/test_encoders.py::TestSimpleMLPEncoder -v

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/multimodal-sensor-fusion-with-attention-saharrgl
plugins: hydra-core-1.3.2, typeguard-4.4.4, langsmith-0.4.41, anyio-4.11.0
collected 2 items                                                              

tests/test_encoders.py::TestSimpleMLPEncoder::test_output_shape PASSED   [ 50%]
tests/test_encoders.py::TestSimpleMLPEncoder::test_gradient_flow PASSED  [100%]

============================== 2 passed in 1.79s ===============================


In [17]:
# Edit src/attention.py
# Test with:
!python -m pytest /content/multimodal-sensor-fusion-with-attention-saharrgl/tests/test_attention.py::TestCrossModalAttention -v

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/multimodal-sensor-fusion-with-attention-saharrgl
plugins: hydra-core-1.3.2, typeguard-4.4.4, langsmith-0.4.41, anyio-4.11.0
collected 3 items                                                              

tests/test_attention.py::TestCrossModalAttention::test_output_shape PASSED [ 33%]
tests/test_attention.py::TestCrossModalAttention::test_with_mask PASSED  [ 66%]
tests/test_attention.py::TestCrossModalAttention::test_gradient_flow PASSED [100%]

============================== 3 passed in 1.81s ===============================


In [18]:
!cd /content/multimodal-sensor-fusion-with-attention-saharrgl/data

!rm -f PAMAP2_Dataset.zip
!rm -rf PAMAP2_Dataset

/bin/bash: line 1: cd: /content/multimodal-sensor-fusion-with-attention-saharrgl/data: No such file or directory


In [19]:
# In a Colab cell
!mkdir -p datasets/pamap2_raw
!cd datasets/pamap2_raw
!wget -nc https://archive.ics.uci.edu/ml/machine-learning-databases/00231/PAMAP2_Dataset.zip
!unzip -n PAMAP2_Dataset.zip >/dev/null
!cd /content

--2025-11-10 18:35:55--  https://archive.ics.uci.edu/ml/machine-learning-databases/00231/PAMAP2_Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘PAMAP2_Dataset.zip’

PAMAP2_Dataset.zip      [     <=>            ] 656.29M  17.4MB/s    in 39s     

2025-11-10 18:36:36 (16.7 MB/s) - ‘PAMAP2_Dataset.zip’ saved [688167640]



In [20]:
!unzip -q -o PAMAP2_Dataset.zip -d data

RAW_DIR = "/content/multimodal-sensor-fusion-with-attention-saharrgl/data/PAMAP2_Dataset/Protocol"
OUT_DIR = "/content/multimodal-sensor-fusion-with-attention-saharrgl/data"

In [21]:
import shutil, os

for split in ["train", "val", "test"]:
    shutil.rmtree("./data/" + split, ignore_errors=True)

In [22]:
import os, re, glob
import numpy as np
import pandas as pd
from collections import Counter
from scipy import interpolate
from pathlib import Path

# ---------------------------------------------------------------------
# Paths
# ---------------------------------------------------------------------
RAW_DIR   = Path("/content/multimodal-sensor-fusion-with-attention-saharrgl/data/PAMAP2_Dataset")
DATA_ROOT = Path("/content/multimodal-sensor-fusion-with-attention-saharrgl/data/pamap2")

# ---------------------------------------------------------------------
# Parameters
# ---------------------------------------------------------------------
TARGET_SR = 100
WIN_SEC   = 5.0
HOP_SEC   = 2.5
WIN       = int(WIN_SEC * TARGET_SR)   # 500
HOP       = int(HOP_SEC * TARGET_SR)   # 250

# Activity IDs to keep (from PAMAP2 protocol)
VALID_IDS    = [1,2,3,4,5,6,7,9,10,11,12,13,16,17,18,19,20,24]
LABEL_TO_IDX = {a: i for i, a in enumerate(VALID_IDS)}

# Make output dirs for splits
for split in ["train", "val", "test"]:
    (DATA_ROOT / split).mkdir(parents=True, exist_ok=True)

# ---------------------------------------------------------------------
# Helper functions
# ---------------------------------------------------------------------
def list_subject_files():
    """Find all subject*.dat files under RAW_DIR (Protocol + Optional)."""
    paths = []
    for folder in ["Protocol", "Optional"]:
        d = RAW_DIR / folder
        if d.is_dir():
            paths += sorted(glob.glob(str(d / "subject*.dat")))
    return paths


def parse_subject_id(path):
    """Extract integer subject ID from filename like 'subject101.dat'."""
    stem = os.path.splitext(os.path.basename(path))[0]
    m = re.search(r"(\d+)", stem)
    return int(m.group(1)) if m else None


def resample_to_100hz(df):
    """
    PAMAP2 columns: 0=time, 1=activityID, 2=HR, others = sensors.
    Resample all columns (except time) to uniform 100 Hz using linear interp.
    """
    ts = df[0].to_numpy(dtype=float)
    if len(ts) < 2 or ts[-1] <= ts[0]:
        return None

    n = int(round((ts[-1] - ts[0]) * TARGET_SR)) + 1
    new_ts = np.linspace(ts[0], ts[-1], n)

    cols = {}
    for col in df.columns:
        if col == 0:
            continue
        f = interpolate.interp1d(
            ts,
            df[col].to_numpy(),
            kind="linear",
            bounds_error=False,
            fill_value="extrapolate",
        )
        cols[col] = f(new_ts)

    out = pd.DataFrame(cols, index=new_ts)
    # new columns: 1=activityID, 2=HR, 3..=sensors
    return out


def window_indices(T, win, hop):
    """Return list of (start, end) indices for sliding windows."""
    return [(s, s + win) for s in range(0, max(0, T - win + 1), hop)]

# ---------------------------------------------------------------------
# Collect windows from all subjects
# ---------------------------------------------------------------------
X_hand, X_chest, X_ankle, X_hr, Y, SUBS = [], [], [], [], [], []

files = list_subject_files()
print(f"Found {len(files)} PAMAP2 files")

for f in files:
    sid = parse_subject_id(f)
    if sid is None:
        print("Skipping unparseable filename:", f)
        continue

    # Read raw file
    df_raw = pd.read_csv(f, sep=r"\s+", header=None, na_values=["NaN"])

    # Keep only rows with valid activities
    df_raw = df_raw[df_raw[1].isin(VALID_IDS)].reset_index(drop=True)
    if len(df_raw) < WIN:
        continue

    df = resample_to_100hz(df_raw)
    if df is None or len(df) < WIN:
        continue

    # ---- Column mapping for PAMAP2 ----
    # 1 = activityID
    # 2 = HR
    # 3-19  = hand sensor (17 features)
    # 20-36 = chest sensor (17 features)
    # 37-53 = ankle sensor (17 features)
    labels = df.loc[:, 1].to_numpy(dtype=int)                # (T,)
    hr     = df.loc[:, 2].to_numpy(dtype=float).reshape(-1, 1)  # (T,1)
    hand   = df.loc[:, 3:19].to_numpy(dtype=float)           # (T,17)
    chest  = df.loc[:, 20:36].to_numpy(dtype=float)          # (T,17)
    ankle  = df.loc[:, 37:53].to_numpy(dtype=float)          # (T,17)

    # Replace NaNs
    hand  = np.nan_to_num(hand)
    chest = np.nan_to_num(chest)
    ankle = np.nan_to_num(ankle)
    hr    = np.nan_to_num(hr)

    # --- Sliding windows ---
    for s, e in window_indices(len(df), WIN, HOP):
        if e - s != WIN:
            continue

        y_win = labels[s:e]
        maj = Counter(y_win).most_common(1)[0][0]
        if maj not in LABEL_TO_IDX:
            continue

        # IMU windows: (WIN, 17)
        X_hand.append(hand[s:e].astype(np.float32))
        X_chest.append(chest[s:e].astype(np.float32))
        X_ankle.append(ankle[s:e].astype(np.float32))

        # HR window: keep as (WIN, 1)
        hr_win = hr[s:e].astype(np.float32)          # (WIN, 1)
        X_hr.append(hr_win)

        Y.append(LABEL_TO_IDX[maj])
        SUBS.append(sid)

# Stack all
X_hand  = np.stack(X_hand).astype(np.float32) if X_hand  else np.empty((0, WIN, 17), np.float32)
X_chest = np.stack(X_chest).astype(np.float32) if X_chest else np.empty((0, WIN, 17), np.float32)
X_ankle = np.stack(X_ankle).astype(np.float32) if X_ankle else np.empty((0, WIN, 17), np.float32)
X_hr    = np.stack(X_hr).astype(np.float32)    if X_hr    else np.empty((0, WIN, 1),  np.float32)

Y    = np.asarray(Y,    dtype=np.int64)
SUBS = np.asarray(SUBS, dtype=np.int32)

print("Collected shapes:")
print("  hand:",  X_hand.shape)
print("  chest:", X_chest.shape)
print("  ankle:", X_ankle.shape)
print("  hr:",    X_hr.shape)          # should be (N, 500, 1)
print("  labels:", Y.shape)
print("  subjects:", np.unique(SUBS))

# (Optional) save full arrays as reference
np.save(DATA_ROOT / "imu_hand.npy",   X_hand)
np.save(DATA_ROOT / "imu_chest.npy",  X_chest)
np.save(DATA_ROOT / "imu_ankle.npy",  X_ankle)
np.save(DATA_ROOT / "heart_rate.npy", X_hr)
np.save(DATA_ROOT / "labels.npy",     Y)
np.save(DATA_ROOT / "subjects.npy",   SUBS)

# ============================================================
# SUBJECT-WISE SPLIT (no leakage across train/val/test)
# ============================================================

all_subjects = sorted(np.unique(SUBS).tolist())
print("All subjects:", all_subjects)

# You can change this split if you want
train_subjects = [101, 102, 103, 104, 105, 106]
val_subjects   = [107]
test_subjects  = [108, 109]

print("Train subjects:", train_subjects)
print("Val subjects:  ", val_subjects)
print("Test subjects: ", test_subjects)

def indices_for_subjects(subject_list):
    subject_list = np.asarray(subject_list)
    mask = np.isin(SUBS, subject_list)
    return np.where(mask)[0]

train_idx = indices_for_subjects(train_subjects)
val_idx   = indices_for_subjects(val_subjects)
test_idx  = indices_for_subjects(test_subjects)

print(f"Train windows: {len(train_idx)}")
print(f"Val   windows: {len(val_idx)}")
print(f"Test  windows: {len(test_idx)}")

def save_split_by_indices(name, indices):
    indices = np.asarray(indices)
    out_dir = DATA_ROOT / name
    out_dir.mkdir(parents=True, exist_ok=True)

    np.save(out_dir / "imu_hand.npy",   X_hand[indices])
    np.save(out_dir / "imu_chest.npy",  X_chest[indices])
    np.save(out_dir / "imu_ankle.npy",  X_ankle[indices])
    np.save(out_dir / "heart_rate.npy", X_hr[indices])   # shape: (N, 500, 1)
    np.save(out_dir / "labels.npy",     Y[indices])
    np.save(out_dir / "subjects.npy",   SUBS[indices])

    print(f"{name}: X={len(indices)} windows -> {out_dir}")

save_split_by_indices("train", train_idx)
save_split_by_indices("val",   val_idx)
save_split_by_indices("test",  test_idx)

# Quick sanity check: shapes + unique labels + unique subjects per split
for split in ["train", "val", "test"]:
    p   = DATA_ROOT / split
    xr  = np.load(p / "heart_rate.npy")
    xh  = np.load(p / "imu_hand.npy")
    y   = np.load(p / "labels.npy")
    subs_split = np.load(p / "subjects.npy")
    print(
        split, "→ hand", xh.shape,
        "hr", xr.shape,
        "labels", y.shape,
        "unique labels:", sorted(set(y.tolist())),
        "unique subjects:", sorted(np.unique(subs_split).tolist())
    )

Found 14 PAMAP2 files
Collected shapes:
  hand: (14159, 500, 17)
  chest: (14159, 500, 17)
  ankle: (14159, 500, 17)
  hr: (14159, 500, 1)
  labels: (14159,)
  subjects: [101 102 103 104 105 106 107 108 109]
All subjects: [101, 102, 103, 104, 105, 106, 107, 108, 109]
Train subjects: [101, 102, 103, 104, 105, 106]
Val subjects:   [107]
Test subjects:  [108, 109]
Train windows: 10086
Val   windows: 1192
Test  windows: 2881
train: X=10086 windows -> /content/multimodal-sensor-fusion-with-attention-saharrgl/data/pamap2/train
val: X=1192 windows -> /content/multimodal-sensor-fusion-with-attention-saharrgl/data/pamap2/val
test: X=2881 windows -> /content/multimodal-sensor-fusion-with-attention-saharrgl/data/pamap2/test
train → hand (10086, 500, 17) hr (10086, 500, 1) labels (10086,) unique labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] unique subjects: [101, 102, 103, 104, 105, 106]
val → hand (1192, 500, 17) hr (1192, 500, 1) labels (1192,) unique labels: [0, 1, 2, 3

In [23]:
import os, glob, shutil
import numpy as np
from pathlib import Path
from sklearn.model_selection import StratifiedShuffleSplit

# ---------------------------------------------------------------------
# Paths (adapted to your repo)
# ---------------------------------------------------------------------
OLD_DATA_ROOT = Path("/content/multimodal-sensor-fusion-with-attention-saharrgl/data/pamap2")
NEW_DATA_ROOT = Path("/content/multimodal-sensor-fusion-with-attention-saharrgl/data/pamap2_resplit")

MODALITIES = ["imu_hand", "imu_chest", "imu_ankle", "heart_rate"]
SEED = 42

def load_split(root: Path, split: str):
    """Load X_{mods}, y for a given split from root/{split}/*.npy."""
    X = {}
    split_dir = root / split
    for m in MODALITIES:
        f = split_dir / f"{m}.npy"
        if not f.is_file():
            raise FileNotFoundError(f"Missing file: {f}")
        X[m] = np.load(f)
    y_path = split_dir / "labels.npy"
    if not y_path.is_file():
        raise FileNotFoundError(f"Missing labels file: {y_path}")
    y = np.load(y_path)
    return X, y

# 1) Load old splits
splits_old = {}
for split in ["train", "val", "test"]:
    Xs, ys = load_split(OLD_DATA_ROOT, split)
    splits_old[split] = (Xs, ys)

# 2) Concatenate all windows into one pool
for split in ["train", "val", "test"]:
    n = len(splits_old[split][1])
    for m in MODALITIES:
        assert splits_old[split][0][m].shape[0] == n, f"N mismatch in {split}, modality {m}"

X_all = {
    m: np.concatenate(
        [splits_old["train"][0][m],
         splits_old["val"][0][m],
         splits_old["test"][0][m]],
        axis=0
    )
    for m in MODALITIES
}
y_all = np.concatenate(
    [splits_old["train"][1],
     splits_old["val"][1],
     splits_old["test"][1]],
    axis=0
).astype(np.int64)

uniq_labels = sorted(np.unique(y_all).tolist())
print("ALL shapes:", {m: X_all[m].shape for m in MODALITIES},
      "| labels:", y_all.shape,
      "| unique labels:", uniq_labels)

# 3) Stratified 70 / 15 / 15 split on windows
sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.30, random_state=SEED)
train_idx, temp_idx = next(sss1.split(np.zeros_like(y_all), y_all))

y_temp = y_all[temp_idx]
sss2 = StratifiedShuffleSplit(n_splits=1, test_size=0.50, random_state=SEED)
val_rel, test_rel = next(sss2.split(np.zeros_like(y_temp), y_temp))
val_idx  = temp_idx[val_rel]
test_idx = temp_idx[test_rel]

splits_new = {
    "train": train_idx,
    "val":   val_idx,
    "test":  test_idx,
}

# sanity: no class only in test
missing_in_train = set(np.unique(y_all[test_idx]).tolist()) - set(np.unique(y_all[train_idx]).tolist())
assert not missing_in_train, f"Classes in test but not in train: {missing_in_train}"

# 4) Write NEW dataset layout: NEW_DATA_ROOT/{train,val,test}/*.npy
if NEW_DATA_ROOT.exists():
    shutil.rmtree(NEW_DATA_ROOT)

for split, idx in splits_new.items():
    out_dir = NEW_DATA_ROOT / split
    out_dir.mkdir(parents=True, exist_ok=True)

    for m in MODALITIES:
        np.save(out_dir / f"{m}.npy", X_all[m][idx])
    np.save(out_dir / "labels.npy", y_all[idx])

    print(f"{split}: N={len(idx)}, classes={sorted(np.unique(y_all[idx]).tolist())}")

ALL shapes: {'imu_hand': (14159, 500, 17), 'imu_chest': (14159, 500, 17), 'imu_ankle': (14159, 500, 17), 'heart_rate': (14159, 500, 1)} | labels: (14159,) | unique labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
train: N=9911, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
val: N=2124, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
test: N=2124, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]


In [24]:
pip install -U tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 42.1 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires tensorboard~=2.19.0, but you have tensorboard 2.20.0 which is incompatible.


In [25]:
%cd /content/multimodal-sensor-fusion-with-attention-saharrgl

/content/multimodal-sensor-fusion-with-attention-saharrgl


In [57]:
!python src/train.py model.fusion_type=early


Configuration:
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities:
  - imu_hand
  - imu_chest
  - imu_ankle
  - heart_rate
  train_split: 0.7
  val_split: 0.15
  test_split: 0.15
  batch_size: 32
  num_workers: 4
model:
  fusion_type: early
  hidden_dim: 256
  output_dim: 128
  num_heads: 4
  dropout: 0.1
  encoders:
    video:
      type: frame
      input_dim: 512
      temporal_pooling: attention
    imu:
      type: sequence
      input_dim: 64
      encoder_type: lstm
      num_layers: 2
training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.1
evaluation:
  metrics:
  - accuracy
  - f1_macro
  - ece
  - nll
  missing_modality_test: true
  uncertainty_analysis: true
  num_calibration_bins: 15
uncertainty:
  method: dropout
  num_mc_samples: 10

In [58]:
!python src/train.py model.fusion_type=late

Configuration:
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities:
  - imu_hand
  - imu_chest
  - imu_ankle
  - heart_rate
  train_split: 0.7
  val_split: 0.15
  test_split: 0.15
  batch_size: 32
  num_workers: 4
model:
  fusion_type: late
  hidden_dim: 256
  output_dim: 128
  num_heads: 4
  dropout: 0.1
  encoders:
    video:
      type: frame
      input_dim: 512
      temporal_pooling: attention
    imu:
      type: sequence
      input_dim: 64
      encoder_type: lstm
      num_layers: 2
training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.1
evaluation:
  metrics:
  - accuracy
  - f1_macro
  - ece
  - nll
  missing_modality_test: true
  uncertainty_analysis: true
  num_calibration_bins: 15
uncertainty:
  method: dropout
  num_mc_samples: 10


In [59]:
!python /content/multimodal-sensor-fusion-with-attention-saharrgl/src/train.py model.fusion_type=hybrid

Configuration:
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities:
  - imu_hand
  - imu_chest
  - imu_ankle
  - heart_rate
  train_split: 0.7
  val_split: 0.15
  test_split: 0.15
  batch_size: 32
  num_workers: 4
model:
  fusion_type: hybrid
  hidden_dim: 256
  output_dim: 128
  num_heads: 4
  dropout: 0.1
  encoders:
    video:
      type: frame
      input_dim: 512
      temporal_pooling: attention
    imu:
      type: sequence
      input_dim: 64
      encoder_type: lstm
      num_layers: 2
training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.1
evaluation:
  metrics:
  - accuracy
  - f1_macro
  - ece
  - nll
  missing_modality_test: true
  uncertainty_analysis: true
  num_calibration_bins: 15
uncertainty:
  method: dropout
  num_mc_samples: 1

In [29]:
!python -m pytest /content/multimodal-sensor-fusion-with-attention-saharrgl/tests/test_fusion.py::TestFusionInterfaces::test_early_fusion_shape -v

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/multimodal-sensor-fusion-with-attention-saharrgl
plugins: hydra-core-1.3.2, typeguard-4.4.4, langsmith-0.4.41, anyio-4.11.0
collected 1 item                                                               

tests/test_fusion.py::TestFusionInterfaces::test_early_fusion_shape PASSED [100%]

============================== 1 passed in 1.72s ===============================


In [30]:
!python -m pytest /content/multimodal-sensor-fusion-with-attention-saharrgl/tests/test_encoders.py::TestSimpleMLPEncoder -v

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/multimodal-sensor-fusion-with-attention-saharrgl
plugins: hydra-core-1.3.2, typeguard-4.4.4, langsmith-0.4.41, anyio-4.11.0
collected 2 items                                                              

tests/test_encoders.py::TestSimpleMLPEncoder::test_output_shape PASSED   [ 50%]
tests/test_encoders.py::TestSimpleMLPEncoder::test_gradient_flow PASSED  [100%]

============================== 2 passed in 1.76s ===============================


In [31]:
!python -m pytest /content/multimodal-sensor-fusion-with-attention-saharrgl/tests/test_attention.py::TestCrossModalAttention -v

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/multimodal-sensor-fusion-with-attention-saharrgl
plugins: hydra-core-1.3.2, typeguard-4.4.4, langsmith-0.4.41, anyio-4.11.0
collected 3 items                                                              

tests/test_attention.py::TestCrossModalAttention::test_output_shape PASSED [ 33%]
tests/test_attention.py::TestCrossModalAttention::test_with_mask PASSED  [ 66%]
tests/test_attention.py::TestCrossModalAttention::test_gradient_flow PASSED [100%]

============================== 3 passed in 1.78s ===============================


In [32]:
!cp -r /content/multimodal-sensor-fusion-with-attention-saharrgl/runs /content/drive/MyDrive/fusion_runs_backup

In [61]:
!find runs/a2_late_pamap2 -name "*.ckpt"

runs/a2_late_pamap2/checkpoints/epoch=59-val/loss=0.2758.ckpt
runs/a2_late_pamap2/checkpoints/epoch=52-val/loss=0.2684.ckpt
runs/a2_late_pamap2/checkpoints/epoch=57-val/loss=0.2715.ckpt
runs/a2_late_pamap2/checkpoints/last.ckpt


In [62]:
!ls runs/a2_late_pamap2/checkpoints

'epoch=0-val'	'epoch=18-val'	'epoch=29-val'	'epoch=43-val'	'epoch=6-val'
'epoch=10-val'	'epoch=1-val'	'epoch=2-val'	'epoch=45-val'	'epoch=7-val'
'epoch=11-val'	'epoch=20-val'	'epoch=30-val'	'epoch=46-val'	'epoch=8-val'
'epoch=12-val'	'epoch=21-val'	'epoch=31-val'	'epoch=48-val'	'epoch=9-val'
'epoch=13-val'	'epoch=22-val'	'epoch=32-val'	'epoch=4-val'	 last.ckpt
'epoch=14-val'	'epoch=23-val'	'epoch=36-val'	'epoch=52-val'
'epoch=15-val'	'epoch=24-val'	'epoch=3-val'	'epoch=57-val'
'epoch=16-val'	'epoch=25-val'	'epoch=40-val'	'epoch=59-val'
'epoch=17-val'	'epoch=28-val'	'epoch=41-val'	'epoch=5-val'


In [71]:
!python src/eval.py \
  --checkpoint runs/a2_early_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_dir analysis/early

Loading model from: runs/a2_early_pamap2/checkpoints/last.ckpt
Creating dataloaders...

Standard Evaluation
Evaluating: 100% 67/67 [00:01<00:00, 57.34it/s]

Test Accuracy: 0.9336
Test F1 (macro): 0.9287
Test Loss: 0.2557

Computing calibration metrics...
ECE: 0.0304
Results saved to: analysis/early/evaluation_results.json

Evaluation complete!


In [82]:
!python src/eval.py \
  --checkpoint runs/a2_early_pamap2/checkpoints/last.ckpt \
  --output_dir analysis/early \
  --device cpu

Loading model from: runs/a2_early_pamap2/checkpoints/last.ckpt
Creating dataloaders...

Standard Evaluation
Evaluating: 100% 67/67 [00:21<00:00,  3.15it/s]

Test Accuracy: 0.9336
Test F1 (macro): 0.9287
Test Loss: 0.2557

Saved prediction arrays to: analysis/early

Computing calibration metrics...
ECE: 0.0304
Results saved to: analysis/early/evaluation_results.json

Evaluation complete!


In [73]:
!python src/eval.py \
  --checkpoint runs/a2_hybrid_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_dir analysis/hybrid

Loading model from: runs/a2_hybrid_pamap2/checkpoints/last.ckpt
Creating dataloaders...

Standard Evaluation
Evaluating: 100% 67/67 [00:01<00:00, 53.30it/s]

Test Accuracy: 0.9228
Test F1 (macro): 0.9159
Test Loss: 0.2491

Computing calibration metrics...
ECE: 0.0199
Results saved to: analysis/hybrid/evaluation_results.json

Evaluation complete!


In [74]:
!python src/eval.py \
  --checkpoint runs/a2_late_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_dir analysis/late

Loading model from: runs/a2_late_pamap2/checkpoints/last.ckpt
Creating dataloaders...

Standard Evaluation
Evaluating: 100% 67/67 [00:01<00:00, 53.03it/s]

Test Accuracy: 0.9397
Test F1 (macro): 0.9319
Test Loss: 0.2329

Computing calibration metrics...
ECE: 0.0318
Results saved to: analysis/late/evaluation_results.json

Evaluation complete!


In [36]:
import numpy as np
from pathlib import Path

DATA_ROOT = Path("data/pamap2_resplit")

for split in ["train", "val", "test"]:
    y = np.load(DATA_ROOT / split / "labels.npy")
    print(split, "labels shape:", y.shape, "unique labels:", sorted(np.unique(y).tolist()))

train labels shape: (9911,) unique labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
val labels shape: (2124,) unique labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
test labels shape: (2124,) unique labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]


In [37]:
!python /content/multimodal-sensor-fusion-with-attention-saharrgl/src/analysis.py --experiment_dir experiments/ --output_dir analysis/

Generating Analysis Plots



Plot generation complete!
Plots saved to: analysis

Note: To generate attention and calibration plots, call:
  - plot_attention_weights(attention_matrix, modality_names)
  - plot_calibration_diagram(confidences, predictions, labels)


In [38]:
from pathlib import Path
import numpy as np

DATA_ROOT = Path("/content/multimodal-sensor-fusion-with-attention-saharrgl/data/pamap2")

for split in ["train", "val", "test"]:
    p    = DATA_ROOT / split
    xh   = np.load(p / "imu_hand.npy")
    xc   = np.load(p / "imu_chest.npy")
    xa   = np.load(p / "imu_ankle.npy")      # if you’re still using ankle
    xr   = np.load(p / "heart_rate.npy")
    y    = np.load(p / "labels.npy")
    subs = np.load(p / "subjects.npy")

    print(
        split,
        "→ hand", xh.shape,
        "chest", xc.shape,
        "ankle", xa.shape,
        "hr", xr.shape,
        "labels", y.shape,
        "unique labels:", sorted(set(y.tolist())),
        "unique subjects:", np.unique(subs),
    )

train → hand (10086, 500, 17) chest (10086, 500, 17) ankle (10086, 500, 17) hr (10086, 500, 1) labels (10086,) unique labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] unique subjects: [101 102 103 104 105 106]
val → hand (1192, 500, 17) chest (1192, 500, 17) ankle (1192, 500, 17) hr (1192, 500, 1) labels (1192,) unique labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] unique subjects: [107]
test → hand (2881, 500, 17) chest (2881, 500, 17) ankle (2881, 500, 17) hr (2881, 500, 1) labels (2881,) unique labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] unique subjects: [108 109]


In [39]:
!ls config


base.yaml  datasets.yaml  fusion_strategies.yaml


In [40]:
%%bash
cd /content/multimodal-sensor-fusion-with-attention-saharrgl

cat > config/single_chest.yaml <<'EOF'
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities: [imu_chest]      # single modality
  batch_size: 32
  num_workers: 4
  num_classes: 18

model:
  fusion_type: early           # still using early fusion block
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1

  encoders:
    imu_chest:
      encoder_type: lstm
      num_layers: 2

training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0   # important: don't randomly drop the only modality

evaluation:
  metrics: [accuracy, f1_macro, ece, nll]
  missing_modality_test: false

experiment:
  name: a2_single_chest_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3

seed: 42
device: cuda
mixed_precision: false
EOF

In [41]:
!cd /content/multimodal-sensor-fusion-with-attention-saharrgl
!python src/train.py --config-name single_chest

Configuration:
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities:
  - imu_chest
  batch_size: 32
  num_workers: 4
  num_classes: 18
model:
  fusion_type: early
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1
  encoders:
    imu_chest:
      encoder_type: lstm
      num_layers: 2
training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0
evaluation:
  metrics:
  - accuracy
  - f1_macro
  - ece
  - nll
  missing_modality_test: false
experiment:
  name: a2_single_chest_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3
seed: 42
device: cuda
mixed_precision: false

Seed set to 42

Creating dataloaders...
Train batches: 310
Val batches: 67
Test batches: 67

Creating model...
Total parameters: 242,770
Trainable parameters:

In [86]:
!python src/eval.py \
  --checkpoint runs/a2_single_chest_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_dir analysis/single_chest

Loading model from: runs/a2_single_chest_pamap2/checkpoints/last.ckpt
Creating dataloaders...

Standard Evaluation
Evaluating: 100% 67/67 [00:00<00:00, 74.35it/s]

Test Accuracy: 0.8879
Test F1 (macro): 0.8778
Test Loss: 0.5065

Saved prediction arrays to: analysis/single_chest

Computing calibration metrics...
ECE: 0.0519
Results saved to: analysis/single_chest/evaluation_results.json

Evaluation complete!


In [83]:
%%bash
cd /content/multimodal-sensor-fusion-with-attention-saharrgl

cat > config/single_hand.yaml <<'EOF'
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities: [imu_hand]      # single modality
  batch_size: 32
  num_workers: 4
  num_classes: 18

model:
  fusion_type: early           # still using early fusion block
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1

  encoders:
    imu_hand:
      encoder_type: lstm
      num_layers: 2

training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0   # important: don't randomly drop the only modality

evaluation:
  metrics: [accuracy, f1_macro, ece, nll]
  missing_modality_test: false

experiment:
  name: a2_single_hand_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3

seed: 42
device: cuda
mixed_precision: false
EOF

In [84]:
!cd /content/multimodal-sensor-fusion-with-attention-saharrgl
!python src/train.py --config-name single_hand

Configuration:
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities:
  - imu_hand
  batch_size: 32
  num_workers: 4
  num_classes: 18
model:
  fusion_type: early
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1
  encoders:
    imu_hand:
      encoder_type: lstm
      num_layers: 2
training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0
evaluation:
  metrics:
  - accuracy
  - f1_macro
  - ece
  - nll
  missing_modality_test: false
experiment:
  name: a2_single_hand_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3
seed: 42
device: cuda
mixed_precision: false

Seed set to 42

Creating dataloaders...
Train batches: 310
Val batches: 67
Test batches: 67

Creating model...
Total parameters: 242,770
Trainable parameters: 24

In [85]:
!python src/eval.py \
  --checkpoint runs/a2_single_hand_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_dir analysis/single_hand

Loading model from: runs/a2_single_hand_pamap2/checkpoints/last.ckpt
Creating dataloaders...

Standard Evaluation
Evaluating: 100% 67/67 [00:00<00:00, 115.37it/s]

Test Accuracy: 0.8729
Test F1 (macro): 0.8640
Test Loss: 0.4995

Saved prediction arrays to: analysis/single_hand

Computing calibration metrics...
ECE: 0.0414
Results saved to: analysis/single_hand/evaluation_results.json

Evaluation complete!


In [44]:
%%bash
cd /content/multimodal-sensor-fusion-with-attention-saharrgl

cat > config/single_ankle.yaml <<'EOF'
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities: [imu_ankle]      # single modality
  batch_size: 32
  num_workers: 4
  num_classes: 18

model:
  fusion_type: early           # still using early fusion block
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1

  encoders:
    imu_ankle:
      encoder_type: lstm
      num_layers: 2

training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0   # important: don't randomly drop the only modality

evaluation:
  metrics: [accuracy, f1_macro, ece, nll]
  missing_modality_test: false

experiment:
  name: a2_single_ankle_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3

seed: 42
device: cuda
mixed_precision: false
EOF

In [87]:
!cd /content/multimodal-sensor-fusion-with-attention-saharrgl
!python src/train.py --config-name single_ankle

Configuration:
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities:
  - imu_ankle
  batch_size: 32
  num_workers: 4
  num_classes: 18
model:
  fusion_type: early
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1
  encoders:
    imu_ankle:
      encoder_type: lstm
      num_layers: 2
training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0
evaluation:
  metrics:
  - accuracy
  - f1_macro
  - ece
  - nll
  missing_modality_test: false
experiment:
  name: a2_single_ankle_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3
seed: 42
device: cuda
mixed_precision: false

Seed set to 42

Creating dataloaders...
Train batches: 310
Val batches: 67
Test batches: 67

Creating model...
Total parameters: 242,770
Trainable parameters:

In [88]:
!python src/eval.py \
  --checkpoint runs/a2_single_ankle_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_dir analysis/single_ankle

Loading model from: runs/a2_single_ankle_pamap2/checkpoints/last.ckpt
Creating dataloaders...

Standard Evaluation
Evaluating: 100% 67/67 [00:00<00:00, 113.97it/s]

Test Accuracy: 0.8734
Test F1 (macro): 0.8702
Test Loss: 0.4490

Saved prediction arrays to: analysis/single_ankle

Computing calibration metrics...
ECE: 0.0446
Results saved to: analysis/single_ankle/evaluation_results.json

Evaluation complete!


In [89]:
%%bash
cd /content/multimodal-sensor-fusion-with-attention-saharrgl

cat > config/single_heart_rate.yaml <<'EOF'
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities: [heart_rate]      # single modality
  batch_size: 32
  num_workers: 4
  num_classes: 18

model:
  fusion_type: early           # still using early fusion block
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1

  encoders:
    heart_rate:
      encoder_type: lstm
      num_layers: 2

training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0   # important: don't randomly drop the only modality

evaluation:
  metrics: [accuracy, f1_macro, ece, nll]
  missing_modality_test: false

experiment:
  name: a2_single_heart_rate_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3

seed: 42
device: cuda
mixed_precision: false
EOF

In [90]:
!cd /content/multimodal-sensor-fusion-with-attention-saharrgl
!python src/train.py --config-name single_heart_rate

Configuration:
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities:
  - heart_rate
  batch_size: 32
  num_workers: 4
  num_classes: 18
model:
  fusion_type: early
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1
  encoders:
    heart_rate:
      encoder_type: lstm
      num_layers: 2
training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0
evaluation:
  metrics:
  - accuracy
  - f1_macro
  - ece
  - nll
  missing_modality_test: false
experiment:
  name: a2_single_heart_rate_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3
seed: 42
device: cuda
mixed_precision: false

Seed set to 42

Creating dataloaders...
Train batches: 310
Val batches: 67
Test batches: 67

Creating model...
Total parameters: 234,578
Trainable para

In [91]:
!python src/eval.py \
  --checkpoint runs/a2_single_heart_rate_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_dir analysis/single_heart_rate

Loading model from: runs/a2_single_heart_rate_pamap2/checkpoints/last.ckpt
Creating dataloaders...

Standard Evaluation
Evaluating: 100% 67/67 [00:00<00:00, 120.68it/s]

Test Accuracy: 0.1026
Test F1 (macro): 0.0112
Test Loss: 2.7892

Saved prediction arrays to: analysis/single_heart_rate

Computing calibration metrics...
ECE: 0.0023
Results saved to: analysis/single_heart_rate/evaluation_results.json

Evaluation complete!


In [93]:
%%bash
cd /content/multimodal-sensor-fusion-with-attention-saharrgl

cat > config/imu_hand_chest.yaml <<'EOF'
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities: [imu_hand, imu_chest]      # two IMU modalities
  batch_size: 32
  num_workers: 4
  num_classes: 18

model:
  fusion_type: early           # use early fusion over hand + chest
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1

  encoders:
    imu_hand:
      encoder_type: lstm
      num_layers: 2
    imu_chest:
      encoder_type: lstm
      num_layers: 2

training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0   # don't drop either modality for this specific test

evaluation:
  metrics: [accuracy, f1_macro, ece, nll]
  missing_modality_test: false

experiment:
  name: a2_imu_hand_chest_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3

seed: 42
device: cuda
mixed_precision: false
EOF

In [94]:
!cd /content/multimodal-sensor-fusion-with-attention-saharrgl
!python src/train.py --config-name imu_hand_chest

Configuration:
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities:
  - imu_hand
  - imu_chest
  batch_size: 32
  num_workers: 4
  num_classes: 18
model:
  fusion_type: early
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1
  encoders:
    imu_hand:
      encoder_type: lstm
      num_layers: 2
    imu_chest:
      encoder_type: lstm
      num_layers: 2
training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0
evaluation:
  metrics:
  - accuracy
  - f1_macro
  - ece
  - nll
  missing_modality_test: false
experiment:
  name: a2_imu_hand_chest_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3
seed: 42
device: cuda
mixed_precision: false

Seed set to 42

Creating dataloaders...
Train batches: 310
Val batches: 67
Test batc

In [95]:
!python src/eval.py \
  --checkpoint runs/a2_imu_hand_chest_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_dir analysis/imu_hand_ches

Loading model from: runs/a2_imu_hand_chest_pamap2/checkpoints/last.ckpt
Creating dataloaders...

Standard Evaluation
Evaluating: 100% 67/67 [00:00<00:00, 85.77it/s] 

Test Accuracy: 0.4948
Test F1 (macro): 0.4175
Test Loss: 1.6136

Saved prediction arrays to: analysis/imu_hand_ches

Computing calibration metrics...
ECE: 0.0203
Results saved to: analysis/imu_hand_ches/evaluation_results.json

Evaluation complete!


In [96]:
%%bash
cd /content/multimodal-sensor-fusion-with-attention-saharrgl

cat > config/imu_hand_chest_hr.yaml <<'EOF'
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities: [imu_hand, imu_chest, heart_rate]
  batch_size: 32
  num_workers: 4
  num_classes: 18

model:
  fusion_type: early            # still using early fusion
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1

  encoders:
    imu_hand:

      encoder_type: lstm

      num_layers: 2
    imu_chest:

      encoder_type: lstm

      num_layers: 2
    heart_rate:

      encoder_type: lstm

      num_layers: 2

training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0   # disable dropping any modality here

evaluation:
  metrics: [accuracy, f1_macro, ece, nll]
  missing_modality_test: false

experiment:
  name: a2_imu_hand_chest_hr_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3

seed: 42
device: cuda
mixed_precision: false
EOF

In [97]:
!cd /content/multimodal-sensor-fusion-with-attention-saharrgl && \
python src/train.py --config-name imu_hand_chest_hr

Configuration:
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities:
  - imu_hand
  - imu_chest
  - heart_rate
  batch_size: 32
  num_workers: 4
  num_classes: 18
model:
  fusion_type: early
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1
  encoders:
    imu_hand:
      encoder_type: lstm
      num_layers: 2
    imu_chest:
      encoder_type: lstm
      num_layers: 2
    heart_rate:
      encoder_type: lstm
      num_layers: 2
training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0
evaluation:
  metrics:
  - accuracy
  - f1_macro
  - ece
  - nll
  missing_modality_test: false
experiment:
  name: a2_imu_hand_chest_hr_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3
seed: 42
device: cuda
mixed_precision: false

Seed 

In [98]:
!python src/eval.py \
  --checkpoint runs/a2_imu_hand_chest_hr_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_dir analysis/imu_hand_chest_hr

Loading model from: runs/a2_imu_hand_chest_hr_pamap2/checkpoints/last.ckpt
Creating dataloaders...

Standard Evaluation
Evaluating: 100% 67/67 [00:01<00:00, 66.41it/s]

Test Accuracy: 0.8988
Test F1 (macro): 0.8908
Test Loss: 0.3737

Saved prediction arrays to: analysis/imu_hand_chest_hr

Computing calibration metrics...
ECE: 0.0389
Results saved to: analysis/imu_hand_chest_hr/evaluation_results.json

Evaluation complete!


In [99]:
%%bash
cd /content/multimodal-sensor-fusion-with-attention-saharrgl

cat > config/imu_chest_ankle_hr.yaml <<'EOF'
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities: [imu_chest, imu_ankle, heart_rate]
  batch_size: 32
  num_workers: 4
  num_classes: 18

model:
  fusion_type: early            # early fusion for 3 modalities
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1

  encoders:
    imu_chest:

      encoder_type: lstm

      num_layers: 2
    imu_ankle:

      encoder_type: lstm

      num_layers: 2
    heart_rate:

      encoder_type: lstm

      num_layers: 2

training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0   # disable random modality dropping

evaluation:
  metrics: [accuracy, f1_macro, ece, nll]
  missing_modality_test: false

experiment:
  name: a2_imu_chest_ankle_hr_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3

seed: 42
device: cuda
mixed_precision: false
EOF

In [100]:
!cd /content/multimodal-sensor-fusion-with-attention-saharrgl && \
python src/train.py --config-name imu_chest_ankle_hr

Configuration:
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities:
  - imu_chest
  - imu_ankle
  - heart_rate
  batch_size: 32
  num_workers: 4
  num_classes: 18
model:
  fusion_type: early
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1
  encoders:
    imu_chest:
      encoder_type: lstm
      num_layers: 2
    imu_ankle:
      encoder_type: lstm
      num_layers: 2
    heart_rate:
      encoder_type: lstm
      num_layers: 2
training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0
evaluation:
  metrics:
  - accuracy
  - f1_macro
  - ece
  - nll
  missing_modality_test: false
experiment:
  name: a2_imu_chest_ankle_hr_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3
seed: 42
device: cuda
mixed_precision: false

Se

In [101]:
!python src/eval.py \
  --checkpoint runs/a2_imu_chest_ankle_hr_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_dir analysis/imu_chest_ankle_hr

Loading model from: runs/a2_imu_chest_ankle_hr_pamap2/checkpoints/last.ckpt
Creating dataloaders...

Standard Evaluation
Evaluating: 100% 67/67 [00:00<00:00, 69.21it/s]

Test Accuracy: 0.9049
Test F1 (macro): 0.8990
Test Loss: 0.3708

Saved prediction arrays to: analysis/imu_chest_ankle_hr

Computing calibration metrics...
ECE: 0.0316
Results saved to: analysis/imu_chest_ankle_hr/evaluation_results.json

Evaluation complete!


In [102]:
%%bash
cd /content/multimodal-sensor-fusion-with-attention-saharrgl

cat > config/imu_hand_chest_ankle.yaml <<'EOF'
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities: [imu_hand, imu_chest, imu_ankle]
  batch_size: 32
  num_workers: 4
  num_classes: 18

model:
  fusion_type: early            # early fusion for 3 modalities
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1

  encoders:
    imu_hand:
      encoder_type: lstm
      num_layers: 2
    imu_chest:
      encoder_type: lstm
      num_layers: 2
    imu_ankle:
      encoder_type: lstm
      num_layers: 2

training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0   # disable random modality dropping

evaluation:
  metrics: [accuracy, f1_macro, ece, nll]
  missing_modality_test: false

experiment:
  name: a2_imu_hand_chest_ankle_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3

seed: 42
device: cuda
mixed_precision: false
EOF

In [103]:
!cd /content/multimodal-sensor-fusion-with-attention-saharrgl && \
python src/train.py --config-name imu_hand_chest_ankle

Configuration:
dataset:
  name: pamap2
  data_dir: ./data/pamap2_resplit
  modalities:
  - imu_hand
  - imu_chest
  - imu_ankle
  batch_size: 32
  num_workers: 4
  num_classes: 18
model:
  fusion_type: early
  hidden_dim: 128
  output_dim: 64
  num_heads: 2
  dropout: 0.1
  encoders:
    imu_hand:
      encoder_type: lstm
      num_layers: 2
    imu_chest:
      encoder_type: lstm
      num_layers: 2
    imu_ankle:
      encoder_type: lstm
      num_layers: 2
training:
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  optimizer: adamw
  scheduler: cosine
  warmup_epochs: 5
  gradient_clip_norm: 1.0
  early_stopping_patience: 10
  augmentation:
    temporal_jitter: 0.1
    gaussian_noise: 0.05
    modality_dropout: 0.0
evaluation:
  metrics:
  - accuracy
  - f1_macro
  - ece
  - nll
  missing_modality_test: false
experiment:
  name: a2_imu_hand_chest_ankle_pamap2
  save_dir: ./runs
  log_every_n_steps: 50
  save_top_k: 3
seed: 42
device: cuda
mixed_precision: false

Seed

In [104]:
!python src/eval.py \
  --checkpoint runs/a2_imu_hand_chest_ankle_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_dir analysis/imu_hand_chest_ankle

Loading model from: runs/a2_imu_hand_chest_ankle_pamap2/checkpoints/last.ckpt
Creating dataloaders...

Standard Evaluation
Evaluating: 100% 67/67 [00:00<00:00, 68.04it/s]

Test Accuracy: 0.9124
Test F1 (macro): 0.9004
Test Loss: 0.2940

Saved prediction arrays to: analysis/imu_hand_chest_ankle

Computing calibration metrics...
ECE: 0.0355
Results saved to: analysis/imu_hand_chest_ankle/evaluation_results.json

Evaluation complete!


In [105]:
import json
from pathlib import Path

results = {
    "dataset": "pamap2",
    "modality_subsets": {
        "all": {
            "accuracy": 0.93,
            "f1_macro": 0.92
        },
        "imu_only": {
            "accuracy": 0.91,   # from your IMU-only run
            "f1_macro": 0.9004
        },
        "heart_only": {
            "accuracy": 0.10,   # from heart_rate-only run
            "f1_macro": 0.01
        }
    }
}

out_path = Path("experiments")
out_path.mkdir(parents=True, exist_ok=True)
with open(out_path / "missing_modality.json", "w") as f:
    json.dump(results, f, indent=2)

print("Wrote experiments/missing_modality.json")

Wrote experiments/missing_modality.json


In [106]:
!cp -r /content/multimodal-sensor-fusion-with-attention-saharrgl/runs /content/drive/MyDrive/fusion_runs_backup

In [107]:
!python src/make_fusion_comparison.py

Saved to experiments/fusion_comparison.json
Saved plot to analysis/fusion_comparison.png


In [109]:
!python src/eval.py \
  --checkpoint runs/a2_hybrid_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_dir analysis/hybrid

Loading model from: runs/a2_hybrid_pamap2/checkpoints/last.ckpt
Creating dataloaders...

Standard Evaluation
Evaluating: 100% 67/67 [00:01<00:00, 53.65it/s]

Test Accuracy: 0.9228
Test F1 (macro): 0.9159
Test Loss: 0.2491

Saved prediction arrays to: analysis/hybrid

Computing calibration metrics...
ECE: 0.0199
Results saved to: analysis/hybrid/evaluation_results.json

Evaluation complete!


In [110]:
!python src/make_uncertainty_report.py

Saved calibration plot to analysis/calibration.png
Saved experiments/uncertainty.json


In [114]:
%%bash
cd /content/multimodal-sensor-fusion-with-attention-saharrgl

cat > src/experiments_uncertainty.py << 'EOF'
"""
Experiment 3: Confidence Calibration Audit

- Loads a trained checkpoint (early / late / hybrid)
- Evaluates on test set
- Computes: ECE, MCE, NLL, Accuracy
- Saves results into: experiments/uncertainty.json
- Saves reliability diagram: analysis/calibration_<fusion>.png
"""

import json
import os
from pathlib import Path

import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm

# --- SAFE GLOBALS FOR TORCH LOAD (same as eval.py) ---
import torch.serialization
import typing
import collections
from omegaconf import DictConfig, ListConfig
from omegaconf.base import ContainerMetadata, Metadata
from omegaconf.nodes import AnyNode

torch.serialization.add_safe_globals([
    DictConfig,
    ListConfig,
    ContainerMetadata,
    Metadata,
    AnyNode,
    typing.Any,
    dict,
    list,
    int,
    float,
    str,
    bool,
    tuple,
    type(None),
    collections.defaultdict,
])
# -----------------------------------------------------

from train import MultimodalFusionModule
from data import create_dataloaders
from uncertainty import CalibrationMetrics


def collect_logits_labels(model, dataloader, device="cpu"):
    model.eval()
    model.to(device)

    all_logits = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Collecting logits for calibration"):
            features, labels, mask = batch
            features = {k: v.to(device) for k, v in features.items()}
            labels = labels.to(device)
            mask = mask.to(device)

            logits = model(features, mask)
            all_logits.append(logits.cpu())
            all_labels.append(labels.cpu())

    all_logits = torch.cat(all_logits)
    all_labels = torch.cat(all_labels)
    return all_logits, all_labels


def main():
    import argparse

    parser = argparse.ArgumentParser(description="Calibration / Uncertainty experiment")
    parser.add_argument("--checkpoint", type=str, required=True,
                        help="Path to model checkpoint")
    parser.add_argument("--output_json", type=str, default="experiments/uncertainty.json",
                        help="Path to JSON file to store calibration results")
    parser.add_argument("--calib_plot", type=str, default=None,
                        help="Path to save reliability diagram (PNG)")
    parser.add_argument("--device", type=str, default="cpu",
                        help="cpu or cuda")
    args = parser.parse_args()

    ckpt_path = args.checkpoint
    print(f"Loading model from: {ckpt_path}")
    model = MultimodalFusionModule.load_from_checkpoint(ckpt_path)
    model.eval()
    model.to(args.device)

    # get config to rebuild dataloader
    config = model.config
    dataset_cfg = config.dataset

    print("Creating dataloaders...")
    _, _, test_loader = create_dataloaders(
        dataset_name=dataset_cfg.name,
        data_dir=dataset_cfg.data_dir,
        modalities=dataset_cfg.modalities,
        batch_size=dataset_cfg.batch_size,
        num_workers=dataset_cfg.num_workers,
    )

    # 1) collect logits + labels
    logits, labels = collect_logits_labels(model, test_loader, device=args.device)
    probs = F.softmax(logits, dim=1)
    confidences, preds = torch.max(probs, dim=1)

    # 2) compute metrics
    accuracy = (preds == labels).float().mean().item()
    ece = CalibrationMetrics.expected_calibration_error(
        confidences, preds, labels, num_bins=15
    )
    mce = CalibrationMetrics.maximum_calibration_error(
        confidences, preds, labels, num_bins=15
    )
    nll = CalibrationMetrics.negative_log_likelihood(logits, labels)

    print("\n=== Calibration / Uncertainty Metrics ===")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"ECE     : {ece:.4f}")
    print(f"MCE     : {mce:.4f}")
    print(f"NLL     : {nll:.4f}")

    # 3) save reliability diagram
    fusion_type = config.model.fusion_type
    if args.calib_plot is None:
        calib_plot_path = Path("analysis") / f"calibration_{fusion_type}.png"
    else:
        calib_plot_path = Path(args.calib_plot)
    calib_plot_path.parent.mkdir(parents=True, exist_ok=True)

    CalibrationMetrics.reliability_diagram(
        confidences.numpy(),
        preds.numpy(),
        labels.numpy(),
        num_bins=15,
        save_path=str(calib_plot_path),
    )
    print(f"Reliability diagram saved to: {calib_plot_path}")

    # 4) update experiments/uncertainty.json
    out_json = Path(args.output_json)
    out_json.parent.mkdir(parents=True, exist_ok=True)

    if out_json.is_file():
        with open(out_json, "r") as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                # if file is corrupted or empty, start fresh
                data = {}
    else:
        data = {}

    # ensure basic structure
    if "dataset" not in data:
        data["dataset"] = dataset_cfg.name
    if "results" not in data or not isinstance(data["results"], dict):
        data["results"] = {}

    # insert / overwrite for this fusion type
    data["results"][fusion_type] = {
        "accuracy": accuracy,
        "ece": ece,
        "mce": mce,
        "nll": nll,
    }

    with open(out_json, "w") as f:
        json.dump(data, f, indent=2)

    print(f"Updated calibration results in: {out_json}")


if __name__ == "__main__":
    main()
EOF

In [115]:
!cd /content/multimodal-sensor-fusion-with-attention-saharrgl

!python src/experiments_uncertainty.py \
  --checkpoint runs/a2_early_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_json experiments/uncertainty.json

Loading model from: runs/a2_early_pamap2/checkpoints/last.ckpt
Creating dataloaders...

=== Calibration / Uncertainty Metrics ===
Accuracy: 0.9336
ECE     : 0.0304
MCE     : 0.2838
NLL     : 0.2581
Reliability diagram saved to: analysis/calibration_early.png
Updated calibration results in: experiments/uncertainty.json


In [116]:
!python src/experiments_uncertainty.py \
  --checkpoint runs/a2_late_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_json experiments/uncertainty.json

Loading model from: runs/a2_late_pamap2/checkpoints/last.ckpt
Creating dataloaders...

=== Calibration / Uncertainty Metrics ===
Accuracy: 0.9397
ECE     : 0.0318
MCE     : 0.3063
NLL     : 0.2339
Reliability diagram saved to: analysis/calibration_late.png
Updated calibration results in: experiments/uncertainty.json


In [117]:
!python src/experiments_uncertainty.py \
  --checkpoint runs/a2_hybrid_pamap2/checkpoints/last.ckpt \
  --device cuda \
  --output_json experiments/uncertainty.json

Loading model from: runs/a2_hybrid_pamap2/checkpoints/last.ckpt
Creating dataloaders...

=== Calibration / Uncertainty Metrics ===
Accuracy: 0.9228
ECE     : 0.0199
MCE     : 0.2204
NLL     : 0.2511
Reliability diagram saved to: analysis/calibration_hybrid.png
Updated calibration results in: experiments/uncertainty.json
